# Noise mitigation via zero-noise extrapolation

Here we will use the functions developed in `./noise_mitigation_utils.py` to demonstrate how this technique can improve
 results when running on a noisy device.

In [1]:
from noise_mitigation_utils import NoiseMitigation
from utility import *
import numpy as np
import qiskit
import cirq
import tequila as tq

Let's take an ansatz trained in S3 and calculate the expectation value on a noisy device.

In [2]:
xyz_data = get_molecular_data('h2o', geometry=0.75, xyz_format=True)

basis = '6-31g'
active = {'B1':[0,1], 'A1':[2,3]}
h2o = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set = basis, active_orbitals = active)
exact = h2o.compute_energy(method='fci')
hf_reference = hf_occ(2*h2o.n_orbitals, h2o.n_electrons)
ham = h2o.make_hamiltonian()
n_ents = ham.n_qubits
ranked_entangler_groupings = generate_QCC_gradient_groupings(ham.to_openfermion(),
                                                             2*h2o.n_orbitals,
                                                             hf_reference,
                                                             cutoff=1.e-6)
entanglers = get_QCC_entanglers(ranked_entangler_groupings, n_ents, 2*h2o.n_orbitals)

for l in range(1):
    if l == 0:
        qansatz = construct_QMF_ansatz(ham.n_qubits, l)
    else:
        qansatz += construct_QMF_ansatz(ham.n_qubits, l)
    qansatz += construct_QCC_ansatz(entanglers)
E = tq.ExpectationValue(H=ham, U=qansatz)
init = {k : np.random.rand() for k in E.extract_variables()}
noiseless = tq.minimize(objective=E, initial_values=init, method='adam', tol=1.0e-6, print_level=1, maxiter=500)
print('Exact energy: {}'.format(exact))
print('Noiseless energy: {}'.format(noiseless.energy))

There are known issues with some psi4 methods and frozen virtual orbitals. Proceed with fingers crossed for hf.
There are known issues with some psi4 methods and frozen virtual orbitals. Proceed with fingers crossed for fci.
Optimizer: <class 'tequila.optimizers.optimizer_gd.OptimizerGD'> 
backend         : qulacs
samples         : None
save_history    : True
noise           : None

Objective       : 1 expectationvalues
Gradient        : 48 expectationvalues
gradient instr  : None
active variables : 24
Iteration: 1 , Energy: -72.74297380, angles: {0beta_0: 0.517419464651125, 0gamma_0: 0.5357851164964361, 0beta_1: 1.0905754418333538, 0gamma_1: 0.23538157703539928, 0beta_2: 0.24911756274382302, 0gamma_2: 0.8403986466830704, 0beta_3: -0.09607478698766096, 0gamma_3: 0.48964940582870414, 0beta_4: 0.10160483648984753, 0gamma_4: 0.7579687921221454, 0beta_5: 0.36503949615149056, 0gamma_5: 0.4347701709504885, 0beta_6: 0.8146618166111748, 0gamma_6: 0.5253562338134674, 0beta_7: 0.9313281129319975

In [5]:
converged = noiseless.variables
noise = tq.circuit.noise.DepolarizingError(0.05, ham.n_qubits)
mitigation = NoiseMitigation(backend='qiskit', noise_model=noise)
zero_n = mitigation.zero_noise_expansion(H=ham, U=qansatz, variables=converged)
print(zero_n)


-75.14085047797134
